In [7]:
from search_wiki_data import *
from make_gazetteer import *
from datasets.process_multiconer import _is_divider


In [8]:
def write_to_file(file_path, mydict):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(mydict, f, ensure_ascii=False, indent=4)

In [9]:
'''
Script to find most frequent topics when search wikidata for a label (LOC, ADR, drug, ...)
'''
label = "drug"
topic_freq = {}
path_to_train_data = 'datasets/vimq/vimq'
lang = 'en'
limit = 5
out_put_file_path = f"retrived_topics/vimq/{label}_topic_freq.txt"

with open(path_to_train_data, 'r', encoding='utf-8') as fin:
    entity_counter = 1
    sleep_time = 300
    entity_counter = 1
    total_lines = sum(1 for _ in fin)
    fin.seek(0)
    for line in tqdm(fin, total=total_lines):
        line = line.strip().replace('\u200d', '').replace('\u200c', '').replace('\u200b', '')
        if _is_divider(line):
            continue
        if line.startswith('# id'):
            continue
        tag = line.split()[-1]
        entity = ' '.join(line.split()[:-1])
        if tag != label:
            continue

        entity_counter += 1

        wiki_topics = get_topics_from_wkdt_search_tool(url, api_url, entity, lang, limit)
        for topic, topic_id in wiki_topics.items():
            text = f"{topic} | {topic_id}"
            topic_freq[text] = topic_freq.get(text, 0) + 1

        if entity_counter % 1000 == 0:
            print(f"Sleep for {sleep_time}!\n")
            time.sleep(sleep_time)
        # write results to file every 100 entities to advoid data loss in case the program crashed
        if entity_counter % 100 == 0:
            sorted_topic_freq = dict(sorted(topic_freq.items(), key=lambda item: item[1], reverse=True))
            write_to_file(out_put_file_path, sorted_topic_freq)
    if entity_counter % 100 != 0:  # This check ensures we only write if there's data that hasn't been written yet
        sorted_topic_freq = dict(sorted(topic_freq.items(), key=lambda item: item[1], reverse=True))
        write_to_file(out_put_file_path, sorted_topic_freq)    
# print(topic_freq)

  0%|          | 0/25369 [00:00<?, ?it/s]

100%|██████████| 25369/25369 [33:41<00:00, 12.55it/s]  
